In [ ]:
!pip install -q tensorflow-addons  # version >= 0.15.0 is required
!pip install -q tensorflow
!pip install -q datasets
!pip install sklearn_crfsuite
!wget -q https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 15.3 MB/s eta 0:00:00


In [ ]:
import copy
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import datasets
from datasets import Dataset
import pandas as pd

import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Load the data from the url
url = "https://drive.google.com/uc?id=1LYi4J9yquBzyVE1op9_uyFLPdo9-jq8t"
train_data = pd.read_csv(url, delimiter='\t', header=None, names=['word', 'tag'])

# Split the dataset into training and testing sets
#msk = np.random.rand(len(data)) < 0.8
#train_data = data[msk]

# Convert the words and tags into numerical representations
word2idx = {w: i+1 for i, w in enumerate(set(train_data['word']))}
tag2idx = {'O': 0, 'B_PRODUCT': 1, 'I_PRODUCT': 2}
train_data['word_idx'] = train_data['word'].map(word2idx)
train_data['tag_idx'] = train_data['tag'].map(tag2idx)

# Create tokens, id, and ner_tags list from the training data
tokens = []
ids = []
ner_tags = []
temp_token_list = []
temp_id_list = []
temp_ner_tag_list = []

# Iterate through the rows of the training data
for index, row in train_data.iterrows():
    # Check if the word column starts with "|"
    if row['word'][0] == '|':
        # Add the temporary lists to the main lists
        if len(temp_token_list)>0:
            tokens.append(temp_token_list)
            ids.append(temp_id_list)
            ner_tags.append(temp_ner_tag_list)
        # Clear the temporary lists
        temp_token_list = []
        temp_id_list = []
        temp_ner_tag_list = []
    else:
        # Add the word, tag, and tag_idx to the temporary lists
        temp_token_list.append(row['word'])
        temp_id_list.append(tag2idx[row['tag']])
        temp_ner_tag_list.append(row['tag'])

# Add the last example to the main lists
if len(temp_token_list)>0:
    tokens.append(temp_token_list)
    ids.append(temp_id_list)
    ner_tags.append(temp_ner_tag_list)

# Split the dataset into train and test datasets
tokens_train, tokens_test, ids_train, ids_test, ner_tags_train, ner_tags_test = train_test_split(
    tokens, ids, ner_tags, test_size=0.2, random_state=42)

# Create a dictionary with 'tokens', 'id', and 'ner_tags' keys for train and test datasets
train_dict = {'tokens': tokens_train, 'id': ids_train, 'ner_tags': ner_tags_train}
test_dict = {'tokens': tokens_test, 'id': ids_test, 'ner_tags': ner_tags_test}

# Create Dataset objects with the train_dict and test_dict dictionaries
train_dataset = Dataset.from_dict(train_dict)
test_data = Dataset.from_dict(test_dict)

In [ ]:
# Print the dataset
print(train_dataset)
print(train_dataset['tokens'][0])
print(train_dataset['id'][0])
print(train_dataset['ner_tags'][0])

# Print the dataset
#print(train_dataset)
print(train_dataset['tokens'][:20])
print(train_dataset['id'][:20])
print(train_dataset['ner_tags'][:20])

Dataset({
    features: ['tokens', 'id', 'ner_tags'],
    num_rows: 26868
})
['CUTTING', 'SET', 'HSS', 'PIE']
[1, 2, 2, 2]
['B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', 'I_PRODUCT']
[['CUTTING', 'SET', 'HSS', 'PIE'], ['JEWELLERY', 'MAKING', 'TOOLS', 'BRASS', 'SHUNK', 'PIN'], ['ETG', 'TOOLS'], ['CARBIDE', 'INSERT', 'CLASS', 'CLASS'], ['UOCHIKYU', 'NEEDLE', 'FILE', 'FLAT', 'FOR', 'MFG', 'PURPOSE'], ['PARCEL', 'PAPER', 'WHITE', 'WHITE'], ['INDUSTRIAL', 'ENGINEERING', 'TOOLS', 'PRECISION'], ['NONMAG', 'CRANKED', 'SOCKET', 'WRENCH'], ['CRUSHER', 'MACHINE', 'SPARES', 'VALVE', 'INSERT'], ['KOL', 'DRILL', 'BIT', 'TOOL', 'HAND', 'TOOLS'], ['HAND', 'TOOLS', 'MICRO', 'FLUSH', 'CUTTING', 'PLIER'], ['CONE', 'OUTLET', 'UPR', 'TGDI'], ['TOOLS', 'CARTRIDGES'], ['ROTARY', 'FILERS', 'HARDWARE', 'TOOLS'], ['HEDGE', 'SHEAR', 'ULTRA', 'TOUCH'], ['DIE', 'CUTTING', 'BLADE', 'DTL', 'SPECIAL', 'LCB', 'ACW', 'COIL', 'COIL', 'ACW'], ['SYNTEK', 'DOCTOR', 'BLADE', 'LENGTH', 'WIDTH', 'THICK'], ['CLAMP', 'FOR', 'MACHINE',

In [ ]:
for item in train_dataset:
  sample_tokens = item['tokens']
  sample_tag_ids = item["id"]
  print(sample_tokens)
  print(sample_tag_ids)
  break

raw_tags = ['O', 'B_PRODUCT', 'I_PRODUCT']
print(raw_tags)
print(type(raw_tags))

sample_tags = [raw_tags[i] for i in sample_tag_ids]

print(sample_tokens)
print(sample_tags)

tags = ['<PAD>'] + raw_tags
print(tags)

TAG_SIZE = len(tags)
VOCAB_SIZE = 20000
EMBEDDING_DIM = 100  # set the embedding dimension to 100

train_tokens = tf.ragged.constant(train_dataset["tokens"])
train_tokens = tf.map_fn(tf.strings.lower, train_tokens)

['CUTTING', 'SET', 'HSS', 'PIE']
[1, 2, 2, 2]
['O', 'B_PRODUCT', 'I_PRODUCT']
<class 'list'>
['CUTTING', 'SET', 'HSS', 'PIE']
['B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', 'I_PRODUCT']
['<PAD>', 'O', 'B_PRODUCT', 'I_PRODUCT']


In [ ]:
# Load the GloVe embeddings file into a dictionary
glove_file = "glove.6B.100d.txt"
word_to_embedding = {}
with open(glove_file, encoding="utf8", mode= "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.asarray(values[1:], dtype="float32")
        word_to_embedding[word] = embedding

In [ ]:
lookup_layer = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=list(word_to_embedding.keys()), mask_token=None
)
embedding_matrix = np.zeros((lookup_layer.vocabulary_size(), EMBEDDING_DIM))
for i, word in enumerate(lookup_layer.get_vocabulary()):
    if word in word_to_embedding:
        embedding_matrix[i] = word_to_embedding[word]

embedding_layer = tf.keras.layers.Embedding(
    lookup_layer.vocabulary_size(), EMBEDDING_DIM, mask_zero=True,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False  # set trainable to False to use the pre-trained embeddings
)

In [ ]:
def create_data_generator(dataset):
  def data_generator():
    for item in dataset:
      yield item['tokens'], item['id']

  return data_generator

data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)
#buraya bak.
train_data = tf.data.Dataset.from_generator(
    create_data_generator(train_dataset),
    output_signature=data_signature
)

In [ ]:
def dataset_preprocess(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens(tokens)

    # increase by 1 for all tag_ids,
    # because `<PAD>` is added as the first element in tags list
    preprocessed_tag_ids = tag_ids + 1

    return preprocessed_tokens, preprocessed_tag_ids


def preprecess_tokens(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)


BATCH_SIZE = 2048

# With `padded_batch()`, each batch may have different length
# shape: (batch_size, None)
train_dataset = (
    train_data.map(dataset_preprocess)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)

In [ ]:
def build_embedding_bilstm_crf_model(
    vocab_size: int, embed_dims: int, lstm_unit: int, tag_size: int
) -> tf.keras.Model:
    x = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="x")

    # Replace the Embedding layer with pre-trained GloVe embedding
    glove_embedding = tf.keras.layers.Embedding(input_dim=lookup_layer.vocab_size(),
                                                output_dim=100,
                                                weights=[embedding_matrix],
                                                trainable=False,
                                                mask_zero=True,
                                                name="glove_embedding")
    y = glove_embedding(x)

    y = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_unit, return_sequences=True)
    )(y)
    decode_sequence, potentials, sequence_length, kernel = tfa.layers.CRF(tag_size)(y)

    return tf.keras.Model(
        inputs=x, outputs=[decode_sequence, potentials, sequence_length, kernel]
    )


In [ ]:
model = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 64, TAG_SIZE)

# preprocess
preprecessd_tokens = preprecess_tokens(sample_tokens)

# expand the tensor to shape: [1, None]. That is add batch dim
inputs = tf.expand_dims(preprecessd_tokens, axis=0)

outputs, *_ = model(inputs)
print(outputs[0])

@tf.function
def crf_loss_func(potentials, sequence_length, kernel, y):
    crf_likelihood, _ = tfa.text.crf_log_likelihood(
        potentials, y, sequence_length, kernel
    )
    # likelihood to loss
    flat_crf_loss = -1 * crf_likelihood
    crf_loss = tf.reduce_mean(flat_crf_loss)

    return crf_loss

optimizer = tf.keras.optimizers.Adam(0.02)

train_loss = tf.keras.metrics.Mean(name="train_loss")

@tf.function(experimental_relax_shapes=True)
def train_step(x, y):
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = model(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)

tf.Tensor([1 2 3 0], shape=(4,), dtype=int32)


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()

    for x, y in train_dataset:
        train_step(x, y)

    print(f"Epoch {epoch + 1}, " f"Loss: {train_loss.result()}")


# print the inputs and expected outputs
print("raw inputs: ", sample_tokens)

# preprocess
preprocessed_inputs = preprecess_tokens(
    sample_tokens
)

Epoch 1, Loss: 3.8536198139190674
Epoch 2, Loss: 2.547311544418335
Epoch 3, Loss: 2.0895702838897705
Epoch 4, Loss: 1.8242772817611694
Epoch 5, Loss: 1.6348438262939453
Epoch 6, Loss: 1.4890105724334717
Epoch 7, Loss: 1.3604776859283447
Epoch 8, Loss: 1.2493884563446045
Epoch 9, Loss: 1.1517715454101562
Epoch 10, Loss: 1.0663516521453857
raw inputs:  ['CUTTING', 'SET', 'HSS', 'PIE']


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

# Prepare test data
test_tokens = tf.ragged.constant(test_data["tokens"])
test_tokens = tf.map_fn(tf.strings.lower, test_tokens)
test_dataset = (
    tf.data.Dataset.from_generator(
        create_data_generator(test_data), output_signature=data_signature
    )
    .map(dataset_preprocess)
    .padded_batch(batch_size=BATCH_SIZE)
)

# Evaluation
y_true = []
y_pred = []

for x, y in test_dataset:
    decoded_sequence, *_ = model.predict(x)
    y_pred.extend(decoded_sequence)
    y_true.extend(y.numpy().tolist())

# Convert tag IDs to tags
y_true = [[tags[i] for i in seq] for seq in y_true]
y_pred = [[tags[i] for i in seq] for seq in y_pred]

# Flatten the sequences
y_true_flat = [tag for seq in y_true for tag in seq]
y_pred_flat = [tag for seq in y_pred for tag in seq]

# Calculate metrics
f1_score = metrics.flat_f1_score(y_true, y_pred, average='weighted', labels=tags)
accuracy = metrics.flat_accuracy_score(y_true, y_pred)
precision = metrics.flat_precision_score(y_true, y_pred, average='weighted', labels=tags)

# Print metrics
print("F1 Score:", f1_score)
print("Accuracy:", accuracy)
print("Precision:", precision)

18/18 [==============================] - 3s 9ms/step
F1 Score: 0.9136733917079655
Accuracy: 0.9163614708947446
Precision: 0.9124278728917394


In [ ]:
sample_tokens_two = ['JEWELLERY', 'MAKING', 'TOOLS', 'WOODEN', 'PIN']
sample_tag_two_ids = [0, 0, 0, 1, 2]

sample_tags_two = [raw_tags[i] for i in sample_tag_two_ids]

print(sample_tokens_two)
print(sample_tags_two)

print("raw inputs: ", sample_tokens_two)

preprocessed_inputs_two = preprecess_tokens(
    sample_tokens_two
)
# expend the batch dim
inputs = tf.reshape(preprocessed_inputs_two, shape=[1, -1])

outputs, *_ = model.predict(inputs)
prediction = [tags[i] for i in outputs[0]]
print(preprocessed_inputs_two)
# Keypoint: EU -> B-ORG, German -> B-MISC, British -> B-MISC
print("ground true tags: ", sample_tags_two)
print("predicted tags: ", prediction)


print(outputs)


['JEWELLERY', 'MAKING', 'TOOLS', 'WOODEN', 'PIN']
['O', 'O', 'O', 'B_PRODUCT', 'I_PRODUCT']
raw inputs:  ['JEWELLERY', 'MAKING', 'TOOLS', 'WOODEN', 'PIN']
1/1 [==============================] - 0s 25ms/step
tf.Tensor([21975   434  4316  4837  8427], shape=(5,), dtype=int64)
ground true tags:  ['O', 'O', 'O', 'B_PRODUCT', 'I_PRODUCT']
predicted tags:  ['O', 'O', 'O', 'B_PRODUCT', 'I_PRODUCT']
[[1 1 1 2 3]]


In [ ]:
# expend the batch dim

inputs = tf.reshape(preprocessed_inputs, shape=[1, -1])

outputs, *_ = model.predict(inputs)
prediction = [tags[i] for i in outputs[0]]
print(preprocessed_inputs)
print("raw inputs: ", sample_tokens)
# Keypoint: EU -> B-ORG, German -> B-MISC, British -> B-MISC
print("ground true tags: ", sample_tags)
print("predicted tags: ", prediction)


print(outputs)

1/1 [==============================] - 0s 33ms/step
tf.Tensor([  2579    209 103169   9247], shape=(4,), dtype=int64)
raw inputs:  ['CUTTING', 'SET', 'HSS', 'PIE']
ground true tags:  ['B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', 'I_PRODUCT']
predicted tags:  ['B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', 'I_PRODUCT']
[[2 3 3 3]]


In [ ]:
Amazon azure icin amazon ile gorusulecek. sanal makine kullanarak fasttext yapabilir miyiz.
Bert embedding ile nasil yapariz bir de bunu deneyecegim.
19 Haziran haftasi tekrar goruselim. En kotu Eylulde 3 ay uzatma talep ederek Aralikta bitirebiliriz, ama hedef eylulde bitirmek.